In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import math
from typing import List, Tuple, Callable

from app.models.race import Race
from app.models.race_entry import RaceEntry
import hyperopt
from app.db.session import SessionLocal
import random
from pydantic import BaseModel
from decimal import Decimal

from app.raceday.bet_strategy.dr_z_eq import get_best_place_show_bets_all

In [14]:
# Track Payback (1 - (track take % / 100))
# This assumes 17% track take
Q = 0.83

# Based on Dr. Z's place-show optimization formula

def get_place_show_all_utility(race: Race, p_l: Callable, s_l: Callable, total_wealth: float) -> float:
    entries: List[RaceEntry] = race.active_entries()

    total: float = 0

    for i in range(len(entries)):
        i_total: float = 0

        for j in range(len(entries)):
            if j == i:
                continue

            j_total: float = 0

            for k in range(len(entries)):
                if k == i or k == j:
                    continue

                k_total: float = 0

                q_i = 1 / entries[i].latest_odds()
                q_j = 1 / entries[j].latest_odds()
                q_k = 1 / entries[k].latest_odds()
                
                # TODO: Is there a better alternative to harville probability?
                harville_prob_place_show = (q_i * q_j * q_k) /  ((1-q_i) * (1-q_i-q_j))
                rebate = calc_rebate(race, entries, place_outlay, show_outlay, i, j, k, total_wealth)
                
                k_total += (harville_prob_place_show * math.log(rebate))
                j_total += k_total

            i_total += j_total

        total += i_total

    return total

def calc_rebate(race: Race, entries: List[RaceEntry], place_outlay: float, show_outlay: float, i: int, j: int, k: int, w0: float) -> float:
    P = race.place_pool_total
    S = race.show_pool_total
    P_i = entries[i].place_pool_total
    P_j = entries[j].place_pool_total
    P_ij = P_i + P_j
    p_i = place_outlay # Could vary
    p_j = place_outlay # Could vary
    p_l = lambda l: place_outlay # Could vary

    s_l = lambda l: show_outlay # Could vary
    s_i = show_outlay
    s_j = show_outlay
    s_k = show_outlay
    S_i = entries[i].show_pool_total
    S_j = entries[j].show_pool_total
    S_k = entries[k].show_pool_total
    S_ijk = S_i + S_j + S_k

    player_place_total_outlay = sum([p_l(i) for i in range(len(entries))])
    place_bet_return = ((Q * (P + player_place_total_outlay)) - (p_i + p_j + P_ij)) / 2
    place_bet_effect = (p_i / (p_i + P_i)) + (p_j / (p_j + P_j))

    player_show_total_outlay = sum([s_l(i) for i in range(len(entries))])
    show_bet_return = ((Q * (S + player_show_total_outlay)) - (s_i + s_j + s_k + S_ijk)) / 3
    show_bet_effect = (s_i / (s_i + S_i)) + (s_j / (s_j + S_j)) + (s_k / (s_k + S_k))

    sum_s_l: float = 0
    for i_2 in range(len(entries)):
        if i_2 == i or i_2 == j or i_2 == k:
            continue
    
        sum_s_l += s_l(i_2)
        
    sum_p_l: float = 0
    for i_2 in range(len(entries)):
        if i_2 == i or i_2 == j:
            continue
            
        sum_p_l += p_l(i_2)
    
    total = (place_bet_return * place_bet_effect) + (show_bet_return * show_bet_effect) + (w0 - sum_s_l - sum_p_l)

    assert total > 0, (
        f"domain error: pl={p_l(0)}; sl={s_l(0)}; "
        f"({place_bet_return} * {place_bet_effect} = {place_bet_return * place_bet_effect})" 
        f" + ({show_bet_return} * {show_bet_effect} = {show_bet_return * show_bet_effect})"
        f" + {w0} - {sum_s_l} - {sum_p_l} = {total}"
    )
    
    return total

def hyperopt_objective(params) -> float:
    expected_utility = get_place_show_all_utility(
        params['race'],
        params['place_outlay'],
        params['show_outlay'],
        params['total_wealth'],
    )

    return 0 - expected_utility


class DrZPlaceShowResult(BaseModel):
    total_outlay: float
    place_outlay: float
    show_outlay: float
    expected_value: float


def get_best_place_show_bets_all(race: Race, max_spend: float) -> DrZPlaceShowResult:
    trials = hyperopt.Trials()

    max_possible_bet = max_spend / len(race.entries) / 2

    # Start value of 2 is based on Keeneland min bet, may vary
    params_space = {
        'race': race,
        'place_outlay': hyperopt.hp.uniform('place_outlay', 2, max_possible_bet),
        'show_outlay': hyperopt.hp.uniform('show_outlay', 2, max_possible_bet),
        'total_wealth': max_spend,
    }


    best = hyperopt.fmin(
        hyperopt_objective,
        space=params_space,
        algo=hyperopt.tpe.suggest,
        max_evals=50,
        trials=trials,
    )

    total_outlay = max_spend
    place_outlay = best['place_outlay']
    show_outlay = best['show_outlay']

    expected_value = get_place_show_all_utility(race, place_outlay, show_outlay, total_outlay)

    return DrZPlaceShowResult(
        total_outlay=total_outlay,
        place_outlay=place_outlay,
        show_outlay=show_outlay,
        expected_value=expected_value,
    )

In [3]:
db = SessionLocal()

In [7]:
races = db.query(Race).all()
race = races[random.randint(0, len(races)-1)]

print(sum([1/entry.latest_odds() for entry in race.active_entries()]))
entries_sorted = race.entries.copy()
entries_sorted.sort(key=lambda x: x.latest_odds())
print(entries_sorted[0])
print(race.win_pool_total, race.place_pool_total, race.show_pool_total)

1.0
<RaceEntry(id=231458,name='Resolution Tree',program_no='10',win_pool_total=273092.1511627907,place_pool_total=273092.1511627907,show_pool_total=273092.1511627907,odds=6.88)>
1878874.0000000002 1878874.0000000002 1878874.0000000002


In [8]:
print(len(race.active_entries()))
best = get_best_place_show_bets_all(race, 100)

15
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 18.64trial/s, best loss: -4.4441284366783025]


In [9]:
print(best)
len(race.active_entries())

total_outlay=100.0 place_outlay=2.048363823265843 show_outlay=2.023942262233154 expected_value=4.4441284366783025


15

In [7]:
from app.raceday.bet_strategy.bet_strategies import PlaceBet, ShowBet, DrZPlaceShowArbBet, BetStrategy, FlatBetOutlayStrategy, AvgCostRewardSortStrategy

In [8]:
DefaultBetStrategy = BetStrategy(
    outlay_strategy=FlatBetOutlayStrategy(), sort_strategy=AvgCostRewardSortStrategy(),
)

In [9]:
pb = PlaceBet(race=race, entries=race.active_entries(), selection=[entries_sorted[0]], strategy=DefaultBetStrategy)

In [10]:
from app.raceday.bet_strategy.bet_strategies import calc_avg_place_reward, calc_place_reward_redux, calc_place_reward, calc_show_reward, calc_avg_show_reward

In [11]:
drzb = DrZPlaceShowArbBet(race=race, entries=race.active_entries(), selection=[race.active_entries()], strategy=DefaultBetStrategy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 30.43trial/s, best loss: -4.532887551122606]


In [12]:
drzb.avg_reward()

471.420305316751

In [13]:
drzb.max_reward()

[<PlaceBet(cost=2.055661112551471, min_reward=0, avg_reward=0.18014380420327905, max_reward=24.23503113889233)>, <PlaceBet(cost=2.055661112551471, min_reward=0, avg_reward=0.6485176951318047, max_reward=20.486941302836918)>, <ShowBet(cost=2.2240020859012506, min_reward=0, avg_reward=0.11123624610985443, max_reward=15.199141090192878)>, <ShowBet(cost=2.2240020859012506, min_reward=0, avg_reward=0.40045048599547595, max_reward=12.104865743984297)>, <ShowBet(cost=2.2240020859012506, min_reward=0, avg_reward=0.1557307445537962, max_reward=11.549412844129343)>]


94.35872060284247

In [25]:
drzb.cost()

120.0